Set up psql connection in python:

In [1]:
import psycopg2
conn = psycopg2.connect(dbname='mergeoruns101717', host='localhost')

In [2]:
cur = conn.cursor()

In [3]:
conn.autocommit = True

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Review of All Tables in MergeoRuns DB
### Identify columns of data to pull into my dataset

### Tables with Columns I want:

* ##### Table: DistanceSorts
    * Pull "Miles" to associate with "Distance" of each race ("Distance" already in PersonEvents)
* ##### Table: EventTypes
    * Pull type name to add against each event ID
* ##### Table: Events
    * Pull Event Date, Mergeo Event, Event Type ID
* ##### Table: PersonEvents
    * Use as starting point for my dataset.  Keep most columns; exclude: Bib, Start, Finish (S/F covered by Time), Phone #, Emergency Contact Phone #, Email, Tshirt Size, Hoodie Size, Notes (blank column)
* ##### Table: Persons
    * Pull Gender
* ##### Table: SS_TeamPeople
    * Pull pre-reg info for a given personID, eventID to put in master dataset; also pull HowHeard column (missing lots of data there though...)
* ##### Table: Series
    * Pull Series name to associate with Series ID
* ##### Table: SeriesEvents
    * Pull Series ID to use for indication of whether given event ID is part of a series or not, and if yes which series


### Tables I won't use (at least for now):

* ##### Table: Cities
    * Pull none; duplicate to data already in PersonEvents table
* ##### Table: EventDistances
    * Pull none; duplicate to data already in PersonEvents table
* ##### Table: Import Database
    * Pull none, all data represented in other tables I'll pull from (DB manager uses this for his import processing to Access)
* ##### Table: SS_Class, SS_Division, SS_Exclude, SS_Hours, SS_TeamReport, SS_Teams
    * Exclude for now; not doing any specific work with scores, sub-categories within a race type at this time.
* ##### Table: SameName
    * pull none; I'm using PersonID, EventID only, not names, so irrelevant for my purposes.
* ##### Tables: SeriesCategories, SeriesPoints, SeriesRules
    * Exclude for now; not doing any specific work with scores / points or sub-categories within a race type at this time.
* ##### Tables: tmpAttendanceCrosstab, tmpHighScores, tmpHighScoresCrosstab, tmpPersonRanks, tmpSeriesEventAttendance, tmpSeriesPoints, tmpSeriesPointsAvg, tmpSeriesRank, tmpSeriesTopXPoints
    * All of these tables are calculations the DB manager did to look at attendance across events, scores, rankings, and to calculate series scores/ranks.  At this point I will exclude all of these.  I will do my own EDA for attendance, and if I do use people's rankings in given events that will be done later (not part of base model).

#### Test run of small subset of data with only the columns I want from PersonEvents:

In [5]:
tst_query = '''
        SELECT "PersonID", "EventID", "Distance", "Time", "Age", "Registration time", "Total fee", "Payment method",
        "Street Address", "City", "State/Province", "Emergency contact name", "Zip Code", "Country", "Contact", 
        "Tshirt", "Hoodie"
        FROM "PersonEvents"
        LIMIT 10;'''

In [6]:
cur.execute(tst_query)

In [7]:
test_output = cur.fetchmany(2)

In [8]:
#test_output

In [9]:
test_df = pd.DataFrame(test_output, columns=["PersonID", "EventID", "Distance", "Time", "Age", "Registration time", "Total fee", "Payment method",
        "Street Address", "City", "State/Province", "Emergency contact name", "Zip Code", "Country", "Contact", 
        "Tshirt", "Hoodie"])

In [10]:
#test_df

#### Look at joins to collect data from all tables that I want and include in combined table:

In [11]:
query = '''
        WITH master_temp
        AS (SELECT pe."PersonID", pe."EventID", pe."Distance", pe."Time", pe."Age", pe."Registration time", 
        pe."Total fee", pe."Payment method", pe."Street Address", pe."City", pe."State/Province", 
        pe."Emergency contact name", pe."Zip Code", pe."Country", pe."Contact", pe."Tshirt", pe."Hoodie", 
        e."Event_Date", e."MergeoEvent", e."EventTypeID", se."SeriesID"
        FROM "PersonEvents" pe
        LEFT JOIN "Events" e ON pe."EventID" = e."EventID"
        LEFT JOIN "SeriesEvents" se ON pe."EventID" = se."EventID") 
        SELECT mt."PersonID", mt."EventID", mt."Age", p."Gender", mt."Distance", ds."Miles", mt."Time", mt."Total fee", 
        sstp."Prereg", mt."Registration time", mt."Payment method", mt."Street Address", mt."City", mt."State/Province",
        mt."Zip Code", mt."Country", mt."Emergency contact name", mt."Contact", mt."Tshirt", mt."Hoodie", 
        mt."Event_Date", mt."MergeoEvent", mt."EventTypeID", et."EventType", mt."SeriesID", s."Series", sstp."HowHeard" 
        FROM master_temp mt
        LEFT JOIN "DistanceSorts" ds ON LOWER(mt."Distance") = LOWER(ds."Distance")
        LEFT JOIN "EventTypes" et ON mt."EventTypeID" = et."EventTypeID"
        LEFT JOIN "Persons" p ON p."PersonID" = mt."PersonID"
        LEFT JOIN "SS_TeamPeople" sstp ON mt."PersonID" = sstp."PersonID" AND mt."EventID" = sstp."EventID"
        LEFT JOIN "Series" s ON mt."SeriesID" = s."SeriesID"
        '''

In [12]:
cur.execute(query)

In [13]:
query_results = cur.fetchall()

In [14]:
type(query_results)

list

#### Create Pandas DF from query results - this will be the base dataset that I'll clean and do feature engineering on

In [70]:
data_columns = ["PersonID", "EventID", "Age", "Gender", "Distance", "Miles", "Time", "Total fee", "SS_Prereg", 
                "Registration time", "Payment method", "Street Address", "City", "State/Province", "Zip Code", 
                "Country", "Emergency contact name", "Contact", "Tshirt", "Hoodie", "Event_Date", "MergeoEvent",
                "EventTypeID", "EventType", "SeriesID", "Series", "HowHeard"]

base_dataset = pd.DataFrame(query_results, columns = data_columns)

In [16]:
#base_dataset.head()

In [17]:
base_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16989 entries, 0 to 16988
Data columns (total 27 columns):
PersonID                  16989 non-null float64
EventID                   16989 non-null int64
Age                       16612 non-null object
Gender                    16972 non-null object
Distance                  14699 non-null object
Miles                     14296 non-null float64
Time                      14691 non-null object
Total fee                 12765 non-null object
SS_Prereg                 813 non-null object
Registration time         11743 non-null datetime64[ns]
Payment method            13491 non-null object
Street Address            14073 non-null object
City                      15243 non-null object
State/Province            16098 non-null object
Zip Code                  14989 non-null object
Country                   15807 non-null object
Emergency contact name    12570 non-null object
Contact                   16434 non-null object
Tshirt              

#### Address person/event entries with missing ages; many of these people have age entries for other races they attended - if they do, will populate with that age, else will use mean age of the dataset

In [18]:
persons_no_age = base_dataset[pd.isnull(base_dataset['Age'])]['PersonID'].values

persIDs_ages = zip(list(base_dataset['PersonID'].values), list(base_dataset['Age'].values))

In [19]:
from collections import defaultdict

D_age = defaultdict(list)

for persID, age in persIDs_ages:
    if pd.notnull(age):
        D_age[persID].append(int(age))

In [20]:
D2_age = defaultdict(list)

for person in persons_no_age:
    ages_list = D_age.get(person)
    if ages_list is None:
        D2_age[person] = np.NaN
    elif len(ages_list) == 0:
        D2_age[person] = np.NaN
    else:
        D2_age[person] = int(np.round(np.mean(ages_list), decimals=0))

In [21]:
base_dataset['Age2'] = base_dataset[['PersonID', 'Age']].apply(lambda row: int(row[1]) if pd.notnull(row[1]) else D2_age[row[0]], axis=1)

In [22]:
base_dataset['Age2'].fillna(value=int(base_dataset['Age2'].mean()), inplace=True)

In [23]:
base_dataset = base_dataset.drop('Age', axis=1)

In [24]:
base_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16989 entries, 0 to 16988
Data columns (total 27 columns):
PersonID                  16989 non-null float64
EventID                   16989 non-null int64
Gender                    16972 non-null object
Distance                  14699 non-null object
Miles                     14296 non-null float64
Time                      14691 non-null object
Total fee                 12765 non-null object
SS_Prereg                 813 non-null object
Registration time         11743 non-null datetime64[ns]
Payment method            13491 non-null object
Street Address            14073 non-null object
City                      15243 non-null object
State/Province            16098 non-null object
Zip Code                  14989 non-null object
Country                   15807 non-null object
Emergency contact name    12570 non-null object
Contact                   16434 non-null object
Tshirt                    4965 non-null object
Hoodie               

#### Address person/event entries with missing genders if these people have gender entries for other races they attended - if they do, will populate with that gender; for remaining will add third value for other

In [25]:
persons_no_gender = base_dataset[pd.isnull(base_dataset['Gender'])]['PersonID'].values

persIDs_genders = zip(list(base_dataset['PersonID'].values), list(base_dataset['Gender'].values))

In [26]:
#persons_no_gender

In [27]:
D_gen = defaultdict(list)

for persID, gender in persIDs_genders:
    if pd.notnull(gender):
        D_gen[persID].append(gender)

In [28]:
D2_gen = defaultdict(list)

for person in persons_no_gender:
    gen = D_gen.get(person)
    if gen is None:
        D2_gen[person] = None
    else:
        D2_gen[person] = gen[0]

In [29]:
#D2_gen

#### In all cases will replace with third value of 'other' based on this

In [30]:
base_dataset['Gender'].fillna(value='Other', inplace=True)

#### Replace missing distances with 'Variable-SS' indicating this is a street scramble and distances vary

In [31]:
base_dataset['Distance'].fillna(value='Variable-SS', inplace=True)

#### Address null values in Miles column - most of these can be populated

In [32]:
base_dataset['Miles'].unique()

array([  6. ,   3. ,   4.4,  13. ,   5. ,   7.5,   8. ,  26. ,  30. ,
         nan,  20. ,  10. ,   4. ,   2.5,   4.2,  50. ,  18. ,  19.5,
         0. ,  63. ])

In [33]:
base_dataset[pd.isnull(base_dataset['Miles'])]['Distance'].unique()

array(['1/2 Marathon', 'Variable-SS', 'Half Marathon early start',
       '5k early start', '5 Mile late start', '10k early start',
       'Half Marathon late start'], dtype=object)

In [34]:
dist_dict = {'1/2 Marathon':13., 'Half Marathon early start':13., '5k early start':3., '5 Mile late start': 5., 
            '10k early start':6., 'Half Marathon late start':13., 'Variable-SS':np.NaN}

In [35]:
base_dataset['Miles2'] = base_dataset[['Distance', 'Miles']].apply(lambda row: row[1] if pd.notnull(row[1]) 
                                                               else dist_dict[row[0]], axis=1)

In [36]:
base_dataset = base_dataset.drop('Miles', axis=1)

#### For now, replaced NANs for times for all street scramble events with 90 minutes; some of these are 3 hr or 2 hr but I don't have the granularity to see that yet.  Follow up question for Dan....

In [37]:
base_dataset['Time2'] = base_dataset[['EventID', 'Time']].apply(lambda row: '1:30:00.0' if row[0] > 999 else row[1], axis=1)

In [38]:
list(base_dataset[pd.isnull(base_dataset['Time2'])].index)

[270, 4115, 4824, 5308, 5879, 6396, 6515, 6700, 7314, 10183, 10186]

In [39]:
#Drop non-street scramble records that are missing times (only 11 of them, most from EventID 14, a couple from 37)
#Check with Dan on these
base_dataset.drop(labels=[270, 4115, 4824, 5308, 5879, 6396, 6515, 6700, 7314, 10183, 10186], inplace=True)

In [40]:
base_dataset = base_dataset.drop('Time', axis=1)

In [41]:
#convert Total fee column from string to float, setting errors to NaNs
base_dataset['Total fee'] = pd.to_numeric(base_dataset['Total fee'], errors='coerce')

In [42]:
#Fill na's in Total fee column with average cost
base_dataset['Total fee'].fillna(value=round(base_dataset['Total fee'].mean(),2), inplace=True)

In [43]:
base_dataset['Preregistered'] = base_dataset[['SS_Prereg', 'Registration time']].apply(lambda row: 1 if ((row[0]) or (pd.notnull(row[1]))) else 0, axis=1)

In [44]:
base_dataset.drop(['SS_Prereg', 'Registration time'], axis=1, inplace=True)

In [45]:
base_dataset['has_emerg_contact'] = base_dataset[['Emergency contact name']].apply(lambda row: 1 if pd.notnull(row[0])
                                                                                  else 0, axis=1)

In [46]:
base_dataset.drop('Emergency contact name', axis = 1, inplace=True)

In [47]:
base_dataset['Tshirt'].fillna(value='N', inplace=True)
base_dataset['Hoodie'].fillna(value='N', inplace=True)

In [71]:
base_dataset.groupby('SeriesID')['SeriesID'].count()

SeriesID
1.0    1993
2.0    2418
3.0     698
4.0    1701
5.0    2261
6.0     792
Name: SeriesID, dtype: int64

In [72]:
#align winter series from 2015/2016 and 2016/2017 under ID 1
base_dataset['SeriesID'] = base_dataset[['SeriesID']].apply(
                                lambda row: 1. if row[0] == 4. else row[0], axis=1)

In [73]:
base_dataset.groupby('SeriesID')['SeriesID'].count()

SeriesID
1.0    3694
2.0    2418
3.0     698
5.0    2261
6.0     792
Name: SeriesID, dtype: int64

In [74]:
#align half marathon series from 206 and 2017 under ID 2
base_dataset['SeriesID'] = base_dataset[['SeriesID']].apply(
                                lambda row: 2. if row[0] == 5. else row[0], axis=1)
#align trail to grill series from 2016 and 2017 under ID 3
base_dataset['SeriesID'] = base_dataset[['SeriesID']].apply(
                                lambda row: 3. if row[0] == 6. else row[0], axis=1)
#assign a series ID to Street Scrambles, set under ID 4
base_dataset['SeriesID'] = base_dataset[[
                                'EventType', 'SeriesID']].apply(
                                lambda row: 4. if row[0] == 'Street Scramble'
                                else row[1], axis=1)

In [75]:
base_dataset.groupby('SeriesID')['SeriesID'].count()

SeriesID
1.0    3694
2.0    4679
3.0    1490
4.0    2287
Name: SeriesID, dtype: int64

In [53]:
base_dataset['SeriesID'].fillna(value=0., inplace=True)

In [54]:
base_dataset['SeriesID'].unique()

array([ 0.])

In [ ]:
base_dataset['HasSeries'] = base_dataset[['SeriesID']].apply(lambda row: 'Y' if pd.notnull(row[0]) else 'N', axis=1)

In [ ]:
base_dataset.drop('Series', axis=1, inplace=True)

In [ ]:
base_dataset.drop('HowHeard', axis=1, inplace=True)

In [ ]:
base_dataset['Miles2'].fillna(value=round(base_dataset['Miles2'].mean(), 0), inplace=True)

In [ ]:
base_dataset['Payment method'] = base_dataset[['Payment method']].apply(lambda row: 'cash' if (
                                                            row[0] == 'Cash' or 
                                                            row[0] == 'cash/comp')
                                                        else row[0], axis=1)

In [ ]:
base_dataset['Payment method'] = base_dataset[['Payment method']].apply(lambda row: 'check' if (
                                                            row[0] == 'Check')
                                                        else row[0], axis=1)

In [ ]:
base_dataset['Payment method'] = base_dataset[['Payment method']].apply(lambda row: 'credit' if (
                                                            row[0] == 'Authorize' or
                                                            row[0] == 'CC' or
                                                            row[0] == 'cc' or
                                                            row[0] == '$261 CC, $30 cash')
                                                        else row[0], axis=1)

In [ ]:
base_dataset['Payment method'] = base_dataset[['Payment method']].apply(lambda row: 'paypal' if (
                                                            row[0] == 'PayPal' or 
                                                            row[0] == 'PayPal pending')
                                                        else row[0], axis=1)

In [ ]:
base_dataset['Payment method'] = base_dataset[['Payment method']].apply(lambda row: 'comp' if (
                                                            row[0] == 'Comp')
                                                        else row[0], axis=1)

In [ ]:
base_dataset['Payment method'] = base_dataset[['Payment method']].apply(lambda row: 'other' if (
                                                            row[0] != 'cash' and row[0] != 'check' and row[0] != 'comp'
                                                            and row[0] != 'credit' and row[0] != 'paypal' and row[0] != None)
                                                        else row[0], axis=1)

In [ ]:
base_dataset['has_pay_method'] = base_dataset[['Payment method']].apply(lambda row: 'N' if row[0] == None else 'Y', axis=1)

In [ ]:
base_dataset.groupby(by='Payment method')['PersonID'].count()

In [ ]:
base_dataset['Contact'].unique()

In [ ]:
base_dataset['Contact'] = base_dataset[['Contact']].apply(lambda row: 'No' if (
                                                            row[0] == 'no' or row[0] == 'NO')
                                                        else row[0], axis=1)

In [ ]:
base_dataset['Contact'] = base_dataset[['Contact']].apply(lambda row: 'No Response' if (
                                                            row[0] == '-1' or row[0] == '0' or row[0] == None)
                                                        else row[0], axis=1)

In [ ]:
base_dataset.groupby('Contact')['Contact'].count()

In [ ]:
base_dataset.info()

In [ ]:
#cur.execute(query)
#for record in cur:
#    print "{}: user {} logged in via {}".format(record[1], record[0], record[2])

In [ ]:
#cur.close() # This is optional
#conn.close() # Closing the connection also closes all cursors